# Struct2XAS examples: from CIF/XYZ to XANES/EXAFS simulations with FDMNES and FEFF

- Authors: Beatriz G. Foschiani and Mauro Rovezzi
- Contact: mauro.rovezzi@esrf.fr
- Last modified: June 2023

Here we provide various examples on the usage of `larch.xrd.struct2xas` module.

## Example 1: Zn K-edge XANES simulation of wurtzite ZnO with FDMNES from a CIF file

This example demonstrates how to use the `Struct2XAS` class to convert a CIF file to a FDMNES XANES input. We will simulate Zn K-edge XAS of ZnO. The experimental data are taken from the SSHADE/FAME database [DOI:10.26302/SSHADE/EXPERIMENT_ST_20180418_001](https://www.sshade.eu/data/EXPERIMENT_ST_20180418_001)

Import the main class and instantiate it with a CIF file and the name of the absorbing element

In [1]:
from larch.xrd.struct2xas import Struct2XAS
mat_obj = Struct2XAS(file = "../structuredata/struct2xas/ZnO_wurtzite_lCode82028.cif", abs_atom="Zn")

[struct2xas] INFO : Structure creation from a CIF file.
[struct2xas] INFO : Frames: 1, Absorbing sites: 1. (Indexes for frames and abs_sites start at 0)


to get the information about absorbing site

In [4]:
# information about absorbing site as pandas.DataFrame
mat_obj.get_abs_sites_info()

idx_abs,specie,frac_coords,wyckoff_site,cart_coords,occupancy_abs_atom,idx_struct
0,Zn2+,"[0.3333, 0.6667, 0.5]",2b,[0. 1.8757 2.6027],1,0


the same, more programmatically

In [6]:
mat_obj.get_abs_sites()

[[0,
  'Zn2+',
  [0.3333, 0.6667, 0.5],
  '2b',
  array([0.    , 1.8757, 2.6027]),
  1,
  0]]

Getters and setters are available for changing the absorber site being considered. We can use the methods `set_abs_site` and `get_abs_site`. This structure has only one absorber site (an example with multiple sites is given later).

In [7]:
mat_obj.set_abs_site(0)

INFO [2023-06-13 12:13:26]: abs_site idx: 0 


to get information on the coordination environment around absorber atom

In [9]:
mat_obj.get_coord_envs_info()

Coord. Env. from absorber atom: Zn at site 0
(For more details use get_coord_envs() method) 


(['T:4', 'T:5'],
   Element  Distance
 0   (O2-)   1.97533
 1   (O2-)   1.97534
 2   (O2-)   1.97534
 3   (O2-)   1.98326
 4   (O2-)   3.22214)

or programmatically

In [10]:
mat_obj.get_coord_envs()


[['Coord. Env. for Site 0',
  [{'ce_symbol': 'T:4',
    'ce_fraction': 0.8411997800440165,
    'csm': 0.012964151865686767,
    'permutation': [0, 1, 2, 3]},
   {'ce_symbol': 'T:5',
    'ce_fraction': 0.15880021995598353,
    'csm': 3.9062261181512445,
    'permutation': [1, 3, 4, 0, 2]}],
  [{'site': PeriodicSite: O2- (0.0000, 1.8757, 0.6194) [0.3333, 0.6667, 0.1190],
    'index': 2,
    'image_cell': array([0, 0, 0])},
   {'site': PeriodicSite: O2- (0.0000, 3.7514, 3.2221) [0.6667, 1.3333, 0.6190],
    'index': 3,
    'image_cell': array([0, 1, 0])},
   {'site': PeriodicSite: O2- (1.6244, 0.9378, 3.2221) [0.6667, 0.3333, 0.6190],
    'index': 3,
    'image_cell': array([0, 0, 0])},
   {'site': PeriodicSite: O2- (-1.6244, 0.9378, 3.2221) [-0.3333, 0.3333, 0.6190],
    'index': 3,
    'image_cell': array([-1,  0,  0])},
   {'site': PeriodicSite: O2- (0.0000, 1.8757, 5.8248) [0.3333, 0.6667, 1.1190],
    'index': 2,
    'image_cell': array([0, 0, 1])}]]]

It is possible to visualize the local environment at a ginen radius from the absorber

In [13]:
mat_obj.visualize(radius=2.5)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

INFO [2023-06-13 13:43:25]: {'Zn': 'red', 'O': 'green'}


To create the FDMNES input file using the default template in a temporary directory

In [18]:
mytemplate = None #: uses default FDMNES XANES template (-> `larch/xrd/templates/fdmnes.tmpl`)
mypath = None #: creates a default structure -> "mydir/fdmnes/input_structure/abs_atom/siteN/"
mat_obj.make_input_fdmnes(radius=7, green=False, template=mytemplate, parent_path=mypath)

INFO [2023-06-13 14:17:17]: written FDMNES input -> /tmp/struct2xas-c97sdofe/fdmnes/ZnO_wurtzite_lCode82028/Zn/site0/job_inp.txt


Run FDMNES (refer to FDMNES documentation if needed)

```python
#Example how to run FDMNES at the ESRF via SLURM 
input = mat_ojb.parent_path
! cd {input}; subfdmnes -c 30
```

The output files created by FDMNES will be:
- `job.txt`: non-convoluted spectra
- `job_conv.txt`: convoluted spectra (by fdmnes with default parameters)
- `job_out.txt`: FDMNES output during the execution of the program (NOTE: this is created by the SLURM submission script)
- `job_sd0.txt`: file with the projected density of states information
- `job_bav.txt`: file with large simulation infos (better to compress it or remove)

Please, refer to [XANES_Convolution.ipynb](./XANES_Convolution.ipynb) for a comparison of the simulated data with experimental one.

### Example 1a: EXAFS simulation with FEFF

We can also simulate the EXAFS spectrum with FEFF

In [2]:
mytemplate = None #: uses default FEFF EXAFS template (-> `larch/xrd/templates/feff_exafs.tmpl`)
mypath = None #: creates a default structure -> "mydir/feff/input_structure/abs_atom/siteN/"
mat_obj.make_input_feff(radius=7, template=mytemplate, parent_path=mypath)

[struct2xas] INFO : written FEFF input in -> /tmp/struct2xas-81rtar3k/feff/ZnO_wurtzite_lCode82028/Zn/site0


run FEFF

In [3]:
#via larch
from larch.xafs import FeffRunner
feff_inp = f"{mat_obj.parent_path}/feff.inp"
sim = FeffRunner(feff_inp)
sim.run()

 : ======== running Feff module feff8l_rdinp ========
 : Feff8L (EXAFS)      release  0.1
 : ZnO_wurtzite_lCode82028
 : 2023-06-13_1627
 : site 0
 : ======== running Feff module feff8l_pot ========
 : Calculating potentials ...
 : free atom potential and density for atom type    0
 : free atom potential and density for atom type    1
 : free atom potential and density for atom type    2
 : initial state energy
 : overlapped potential and density for unique potential    0
 : overlapped potential and density for unique potential    1
 : overlapped potential and density for unique potential    2
 : muffin tin radii and interstitial parameters
 : : ipot, Norman radius, Muffin tin radius, Overlap
 : 0  1.40713E+00  1.28459E+00  1.15000E+00
 : 1  1.39431E+00  1.27947E+00  1.15000E+00
 : 2  1.08364E+00  9.93037E-01  1.15000E+00
 : : mu_old=    -0.892
 : Done with module 1: potentials.
 : ======== running Feff module feff8l_xsph ========
 : Calculating cross-section and phases...
 : absorption

Note: can also be run via direct call to `feff8l`

```
! cd {mat_obj.parent_path}; feff8l
```

load the simulated EXAFS

In [37]:
import glob
xmu = glob.glob(f"{mat_obj.parent_path}/xmu.dat")[0]
from larch.io import read_ascii
gsim = read_ascii(xmu, labels=["energy", "erel", "k", "mu", "mu0", "chi"])

Load experimental data and quickly extract EXAFS

In [38]:
gexp = read_ascii("../fdmnes/ZnO_SSHADE.data.txt", labels=["energy", "mu"])
from larch.xafs import pre_edge, autobk
pre_edge(gexp, e0=9661)
autobk(gexp)

plot the EXAFS. *Note that the experimental EXAFS data stops at 8 $\AA^{-1}$*

In [39]:
from larch.plot.plotly_xafsplots import PlotlyFigure
fig = PlotlyFigure()
fig.add_plot(gsim.k+0.3, gsim.chi*gsim.k**2, label="FEFF simulation")
fig.add_plot(gexp.k, gexp.chi*gexp.k**2, label="EXP data")

fig.set_style(title="Wurtzite ZnO EXAFS", width=800, height=500, xaxis_title="k (Å^-1)", yaxis_title="chi(k) * k^2" )
fig.set_xrange(0, 10)
fig.show()